# EVACUACIÓN DE VALENCIA EN PYTHON

**Grupo 5 Proyecto III**

- Elías Balbaneda Herreros
- Ángela Mira Abad
- Arnau Garcia i Cucó
- Daniel Oliver Belando
- Andreu Simó Vidal
- José Gellida Bayarri

## ESTABLECIMIENTO DE PARÁMETROS

### LIBRERÍAS

Se importarán las siguientes librerías para realizar este proyecto:

- `networkx` para realizar el grafo de la ciudad.
- `matplotlib` para realizar las diferentes visualizaciones.
- `os` para facilitar el acceso a loss archivos
- `pandas` para guardar e importar dataframes.
- `geopandas` para obtener una mayor información a partir de los archivos ".shp" que usemos.
- `numpy` para optimizar diversas labores matemáticas.
- `random` para realizar aproximaciones aleatorias siguiendo una dist. uniforme.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from random import random

### GRAFO INICIAL

El grafo inicial de Valencia lo obtenemos mediante el ".shp2 de los ejes lineales de las calles. con la función `nx.read_shp`, se estimaría el comportamiento de las calles, estableciendo nodos en los diferentes cruces y aristas en los diferentes tramos. Se usaría la distancia euclidea del tramo cómo peso de las aristas, usándolo así como si de difernetes distancis se tratáse.

In [ ]:
G=nx.read_shp('./EJES_CALLES/EJES-CALLE.shp') 
pos = {k: v for k,v in enumerate(G.nodes())}

X=nx.Graph()
X.add_nodes_from(pos.keys())
l=[set(x) for x in G.edges()]
        
edg_mal=[tuple(k for k,v in pos.items() if v in sl) for sl in l]
nx.draw_networkx_nodes(X,pos,node_size=0.01,node_color='r')
edg = []

for n in edg_mal:
    if len(n)>1:
        edg.append((n[0],n[1],round(((pos[n[0]][0] - pos[n[1]][0])**2 + (pos[n[0]][1] - pos[n[1]][1])**2)**(1/2),4)))
   

X.add_weighted_edges_from(edg)
nx.draw_networkx_edges(X,pos)

dic = {}
for n in X.edges.data('weight'):
    dic[(n[0],n[1])] = n[2]
    
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title('Valencia')

### REPARTO DE HABITANTES

En este caso, utilizando `geopandas`, importaríamos los datos del shape file de las poblaciones en las diferentes manzanas. Los polígonos también quedan almacenados en el dataset.

In [ ]:
edificios = gpd.read_file('./MANZANAS_POB/MANZANAS_POB.shp')
edificios = edificios.fillna(0)
edificios.head()

A partir de la siguiente función (`repartir_habitantes`), se obtendría una aproximación de los habitantes que "residen" en cada uno de los nodos. Este atributo serviría para obtener un nodo inicial a partir del cual poder determinar flujos y caminos. devolvería un diccionario con los habitantes correspondientes a cada uno de los nodos.

La función generaría un punto aleatorio dentro del espacio de representación del polígono del edificio. Despues se comprobaría que dicho punto estuviése incluído en dicho polígono, y, por último, encontraría para dicho punto su nodo más próximo, asignándolo a éste último. Este proceso se repetirí apara todos los habitantes de los diferentes edificios de la ciudad.

In [ ]:
def repartir_habitantes(grafo, edificios):

    def punto_aleatorio():
        a = pol.bounds
        ymin, ymax = a[1], a[3]
        xmin, xmax = a[0], a[2]
        y = random()*(ymax-ymin) + ymin
        x = random()*(xmax-xmin) + xmin
        return x,y

    def comprobar_punto(x,y):
        return shp.geometry.Point(x,y).within(pol)
    
    def nodo_mas_cercano(x,y):
        dist_min = 10**10
        nodo = 0
        for k,v in pos.items():
            lon, lat = v
            dist = ((lon-x)**2 + (lat-y)**2)
            if dist < dist_min:
                nodo = k
                dist_min = dist
        return nodo
    
    h = 0
    habitantes = {}
    for i in list(grafo.nodes):
        habitantes[i] = 0
    for j in edificios.index:
        pol = edificios.iloc[j,5]
        hab = int(edificios.iloc[j,4])
        for k in range(hab):
            print (f"Va por {h:.2f} %", end="\r")
            valid = False
            while not valid:
                x,y = punto_aleatorio()
                if comprobar_punto(x,y):
                    n = nodo_mas_cercano(x,y)
                    habitantes[n] += 1
                    valid = True
                    h += 1/7782.37
    return habitantes

En este caso no se ejecutará debido a su alto coste computacional, debido a las diferentes comprobaciones que realiza la función internamente. Posteriormente, se almacenaría el vector con dichos valores.

In [ ]:
## habs = repartir_habitantes(X, edificios)
## habs = np.array(list(habs.values()))
## np.savetxt("habs.csv",habs)

Para modelar este reparto, se obtendría a partir del ".csv" en el que está almacenada dicha información.

In [ ]:
habs = np.array(np.loadtxt("habs.csv"),dtype = np.int32)
habs = list(habs)

### ESTIMACIÓN DE LOS VEHÍCULOS

Para la estimación de los vehículos que circularían en caso de evacuación interpretamos una posibilidaad de que dicho habitante fuése conductor de coche. En este caso, la tasa de coches por habitante sería de 0,2. Además, en dicho caso, se interpretaría que en cada coche son llevadas 3 personas o menos al punto de evacuación, en función de los habitantes restantes en un nodo.

In [ ]:
transportes = []

for i in habs:
    vehiculos = 0
    peatones = 0
    for j in range(i):
        if random() > 0.2:
            peatones += 1
        else:
            if peatones < 2:
                peatones = 0
            else:
                peatones -= 2
            vehiculos += 1
    transportes.append([vehiculos, peatones])

### ESTABLECIMIENTO DE LAS SALIDAS

En este caso, para establecer las salidas serán usadas dos funciones: `ampliar`, para obtener una visión concreta de Valencia en unas coordenadas concretas, y `l_nodos`, que determinaría cuales habrían sido, en aquel caso, los nodos a escoger.

In [ ]:
def ampliar(x,y, X,pos,dif=50):
    nx.draw_networkx_nodes(X,pos,node_size=0.5,node_color='r')
    nx.draw_networkx_edges(X,pos)
    plt.xlim(x-dif, x+dif)
    plt.ylim(y-dif, y+dif)
    
def l_nodos(nodos,x,y,dif=50):
    for n in range(len(nodos)):
        if nodos[n][0]<x+dif and nodos[n][0]>x-dif and nodos[n][1]>y-dif and nodos[n][1]<y+dif:
            print(nodos[n],n)

Aqui se mostraría un ejemplo de cómo funcionan ambas funciones.

In [ ]:
ampliar(721500, 4369000, X,pos,dif=100)
l_nodos(nodos,721500, 4369000,dif=50)

La lista de salidas obtenidas sería la siguiente.

In [ ]:
salidas =[8168,8206,8472,8031,7034,1222,1957,9125,8266,2973,4665,4887,3587,7716,276]

In [ ]:
nodos = G.nodes()
l = [nodos[salidas[n]] for n in range(len(salidas))]
print(l)
nx.draw_networkx_edges(X,pos)

plt.xlim(721000, 736000)
plt.ylim(4350000, 4369000)
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.title('Valencia')
plt.plot([n[0] for n in l],[n[1] for n in l],'ro')

### OBTENCIÓN DE LA COMPONENTE CONEXA DE LA CIUDAD

A la hora de plantear la evacuación tuvimos un gran problema. La ciudad entera, como grafo, no era conexo. Había diversos poblados que, a la hora de representarlos, no estaban conectados con el resto del municipio, sino con otras localidades. Esto era un gran problema a la hora de modelar la evacuación, ya que no podrían acceder a salida alguna estando apartados de la ciudad o del resto de poblaciones.

En este caso, mediante la función `connected_components`, se obtienen las componentes conexas en función de su cantidad de nodos. En este caso, los nodos del núcleo de población de la ciudad sería el primer elemento de dicha iteración.

In [ ]:
vlc_nodos = list(list(nx.connected_components(X))[0])
print(vlc_nodos[:10])

A partir de estos nodos, obtenemos un nuevo grafo, con los nodos y las aristas que corresponden únicamente a la componente de la ciudad de Valencia.

In [ ]:
vlc=nx.Graph()
vlc.add_nodes_from(vlc_nodos)
vlc_edg = []
for v1,v2,dist in edg:
    if v1 in vlc_nodos and v2 in vlc_nodos:
        vlc_edg.append((v1,v2,dist))
vlc.add_weighted_edges_from(vlc_edg)
nx.draw_networkx_edges(vlc,pos)
plt.xlim(721000, 732500)
plt.ylim(4360000, 4381000)
l = [nodos[salidas[n]] for n in range(len(salidas))]
plt.plot([n[0] for n in l],[n[1] for n in l],'ro')

### DISTANCIAS Y CAMINOS A LAS SALIDAS

Para interpretar cómo sería el proceso de evacuación, obtendríamos los diferentes caminos y pesos de cada uno de los nodos para todas las salidas el grafo. Al igual que en el caso de los habitantes, estos datos son almacenados en un ".csv" externo, para futuras ocasiones.

En este caso, se ejecuta Dijkstra para todos los nodos del grafo en relación a todas las salida de éste. Se obtienen pesos y se guardan en un diccionario. A partir de esta lista de diccionarios, se obtiene un dataframe, mediante el cuál se va a trabajar en este aspecto.

In [ ]:
def todos_escapes(salidas,nodos):
    lista = []
    for i in nodos:
        print(i,end = "\r")
        dists = {"nodo":i}
        for s in salidas:
            camino = nx.shortest_path(vlc, i, s, weight="weight")
            dists[str(s)+"_camino"] = camino
            dists[str(s)+"_peso"]  = nx.classes.function.path_weight(vlc, camino, weight="weight")
        lista.append(dists)
    return lista

A partir de los datos previos se crea un ".csv" que almacena estos caminos y estos pesos.

In [ ]:
## escs = todos_escapes(salidas, vlc_nodos)
## escapes = pd.DataFrame(escs)
## escapes.to_csv("escapes.csv",sep = ";")

Posteriormente, el dataframe se obtiene a partir de estos datos.

In [ ]:
escapes = pd.read_csv("escapes.csv",sep = ";")
escapes = escapes.set_index("nodo")
escapes.head()

## MODELOS

### APROXIMACIÓN BÁSICA

Mediante este caso, se obtienen las salidas para cada nodo en base a sus distancias. La salida con una mejor distancia para cada nodo es almacenada en la lista `mejor_salida`.

In [ ]:
columnas = [i for i in escapes.columns if i[-5:] == "_peso"]
pesos = escapes[columnas].to_numpy()
mejor_salida = []
for i in range(np.shape(pesos)[0]):
    dists = pesos[i]
    ind = np.argmin(dists)
    mejor_salida.append((ind,dists[ind]))
print(mejor_salida[:20])

En este caso, se calcula para cada una de las salidas la gente que saldrá en ella con la lista `gente_salida`

In [ ]:
gente_salida = [0]*len(salidas)
i = 0
for k,v in mejor_salida:
    gente_salida[k] += habs[vlc_nodos[i]]
    i += 1
print(gente_salida)

### MODELO FINAL

Para realizar este modelo, primeramente, se obtendrá el `array` `transportes_vlc`, a partir de los transportes obtenidos de los nodos de la ciudad de Valencia.

In [ ]:
transportes_vlc = np.array(transportes)[vlc_nodos]

Posteriormente, se genera la función principal, `salida`. Esta función recibiría la lista de los transportes de la ciudad, el dataframe con los diferentes caminos y pesos de las salidas, y, a su vez, tambiñen la lista de las salidas, una lista de pesos para cada una de las salidas, otra con los carriles para cada una de las salidas, y una cantidad de minutos determinados en los que realizar el análisis.

A partir de estos datos, se obtendría una lista, en la que se aprecian tres estados: agentes evacuados, agentes esperando salir (en cola), y agentes que aún no habrían sido evacuados. Las salidas se asignarían mediante una ponderación de las distancias de cada uno de los surtidores, y obteniendo posteriormente a esta ponderación, las mediciones mínimas. Posteriormente, asignando una velocidad aleatoria para cada uno de los vehiculos y peatones, se determinaría el tiempo que tardarían en llegar a la salida, y, mediante los carriles de las carreteras, el flujo máximo de coches que podrían pasar por minuto por ahí.

El análisis se haría en periodos de minutos.

In [ ]:
def salida(transportes_vlc, escapes, salidas, vector, f_max_coches, minutos = 60):

    columnas = [i for i in escapes.columns if i[-5:] == "_peso"]
    pesos = escapes[columnas].to_numpy()
    vector = np.array(vector)
    
    n_nodos = pesos.shape[0]
    n_salidas = len(salidas)
    
    evacuacion_total = [[] for i in range(n_salidas)]
    
    ev_completa = [-1]*n_salidas
    
    mejor_salida = []
    for i in range(n_nodos):
        dists = pesos[i]
        pond = np.multiply(dists, vector)
        ind = np.argmin(pond)
        mejor_salida.append((ind,dists[ind]))
    
    asig_salida = []
    for i in range(n_salidas):
        asig_salida.append(list())
    for i in range(n_nodos):
        k,v = mejor_salida[i]
        asig_salida[int(k)].append((list(transportes_vlc[i]),v))

    evacuacion = []
    tiempos_salida = []
    
    for i in range(n_salidas):
        lista_i = [[0,0,0],[0,0,0]]
        salidas_i = [[0]*minutos,[0]*minutos]
        for k,v in asig_salida[i]:
            lista_i[0][2] += int(k[0])
            lista_i[1][2] += int(k[1])
            for j in range(int(k[0])):
                t = int((v/(random()*7+7))//60)
                if t < minutos:
                    salidas_i[0][t] += 1
            for j in range(int(k[1])):
                t = int((v/(random()*0.2+1.3))//60)
                if t < minutos:
                    salidas_i[1][t] += 1
        evacuacion.append(lista_i)
        tiempos_salida.append(salidas_i)
    
    def calcular_evacuados(i):
        for j in range(n_salidas):
            ent_v = tiempos_salida[j][0][i]
            evacuacion[j][0][2] -= ent_v
            evacuacion[j][0][1] += ent_v
            flujo_v = min(vehiculos_max[j],evacuacion[j][0][1])
            evacuacion[j][0][1] -= flujo_v
            evacuacion[j][0][0] += flujo_v
            ent_p = tiempos_salida[j][1][i]
            evacuacion[j][1][2] -= ent_p
            evacuacion[j][1][1] += ent_p
            flujo_p = min(160,evacuacion[j][1][1])
            evacuacion[j][1][1] -= flujo_p
            evacuacion[j][1][0] += flujo_p
            evacuacion_total[j].append([i.copy() for i in evacuacion[j]])
    
        for j in range(n_salidas):
            if sum(evacuacion[j][0][1:])+sum(evacuacion[j][1][1:]) == 0 and ev_completa[j] == -1:
                ev_completa[j] = i+1
        
    
    def imprimir_parametros(i):
        cadena = f"MINUTO {i+1}"
        cadena += f"""
--------------------
COCHES: Evacuados   {[evacuacion[j][0][0] for j in range(n_salidas)]}
        En cola     {[evacuacion[j][0][1] for j in range(n_salidas)]}
        Por evacuar {[evacuacion[j][0][2] for j in range(n_salidas)]}
PEATONES: Evacuados {[evacuacion[j][1][0] for j in range(n_salidas)]}
        En cola     {[evacuacion[j][1][1] for j in range(n_salidas)]}
        Por evacuar {[evacuacion[j][1][2] for j in range(n_salidas)]}
        """
        
        print(cadena)

    for i in range(minutos):
        calcular_evacuados(i)
        imprimir_parametros(i)
    print("""TIEMPO PARA EVACUAR SALIDA
--------------------""")
    print(ev_completa)
    return evacuacion_total

Aquí se ejecutaría la evolución del modelo óptimo que obtendríamos.

In [ ]:
n_carriles = [2,1,2,2,0.1,4,0.1,1,4,4,2,4,4,1,4]
vehiculos_max = [i*80 for i in n_carriles]
pesos_salidas = [1.255,1.075,1.12,1.035,4.1,1.48,10.2,0.7,1.595,2.23,0.89,1.395,1.295,2.645,1.505]
evolucion = salida(transportes_vlc, escapes, salidas,pesos_salidas , vehiculos_max, 188)

Las siguientes listas se obtendrán para realizar los gráficos de los estados de los agentes por minutos.

In [ ]:
coches_evacuados = []
coches_en_cola = []
coches_no_evacuados = []
peatones_evacuados = []
peatones_en_cola = []
peatones_no_evacuados = []
for i in range(len(evolucion[0])):
    c_ev = 0
    c_en_cola = 0
    c_no_ev = 0
    p_ev = 0
    p_en_cola = 0
    p_no_ev = 0
    for j in range(len(evolucion)):
        c_ev += evolucion[j][i][0][0]
        c_en_cola += evolucion[j][i][0][1]
        c_no_ev += evolucion[j][i][0][2]
        p_ev += evolucion[j][i][1][0]
        p_en_cola += evolucion[j][i][1][1]
        p_no_ev += evolucion[j][i][1][2]
    coches_evacuados.append(c_ev)
    coches_en_cola.append(c_en_cola)
    coches_no_evacuados.append(c_no_ev)
    peatones_evacuados.append(p_ev)
    peatones_en_cola.append(p_en_cola)
    peatones_no_evacuados.append(p_no_ev)

A partir de estas y de listas auxiliares, harían gráficos de barras apiladas, los cuáles representarían los diferentes estados para los diferentes minutos en el modelo.

In [ ]:
mins = list(range(len(evolucion[0])))
coches_aux = [coches_en_cola[i]+coches_no_evacuados[i] for i in mins]
peatones_aux = [peatones_en_cola[i]+peatones_no_evacuados[i] for i in mins]
fig, ax = plt.subplots()
ax.bar(mins, coches_no_evacuados, label = "Non evacuated", color = "grey")
ax.bar(mins, coches_en_cola, label = "In congestion", color = "gold", bottom = coches_no_evacuados)
ax.bar(mins, coches_evacuados, label = "Evacuated", color = "green", bottom = coches_aux)
ax.set_title("Car evacuation", fontsize = 30)
ax.set_ylabel("Evacuated cars", fontsize = 20)
ax.set_xlabel("Minutes", fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.legend(prop = {"size" : 24}, framealpha = 1)
fig.set_size_inches(24,13.5)
fig.savefig("coches_plt.png")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.bar(mins, peatones_no_evacuados, label = "Non evacuated", color = "grey")
ax.bar(mins, peatones_en_cola, label = "In congestion", color = "gold", bottom = peatones_no_evacuados)
ax.bar(mins, peatones_evacuados, label = "Evacuated", color = "green", bottom = peatones_aux)
ax.set_title("Pedestrian evacuation", fontsize = 30)
ax.set_ylabel("Evacuated pedestrians", fontsize = 20)
ax.set_xlabel("Minutes", fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.legend(prop = {"size" : 24}, framealpha = 1)
fig.set_size_inches(24,13.5)
fig.savefig("peatones_plt.png")
plt.show()

En estos gráficos son representados las diferentes cantidades de las retenciones de peatones y coches.

In [ ]:
fig, ax = plt.subplots()
ax.plot(mins, coches_en_cola, color = "blue", linewidth = 5)
ax.set_title("Cars in traffic congestion", fontsize = 30)
ax.set_ylabel("Cars", fontsize = 20)
ax.set_xlabel("Minutes", fontsize = 20)
ax.set_ylim([0,110000])
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
fig.set_size_inches(24,13.5)
fig.savefig("coches_cola_plt.png")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(mins, peatones_en_cola, color = "blue", linewidth = 5)
ax.set_title("Pedestrians in traffic congestion", fontsize = 30)
ax.set_ylabel("Pedestrians", fontsize = 20)
ax.set_xlabel("Minutes", fontsize = 20)
ax.set_ylim([0,175000])
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
fig.set_size_inches(24,13.5)
fig.savefig("peatones_cola_plt.png")
plt.show()